In [15]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sklearn.metrics import fbeta_score, precision_recall_curve, auc, roc_curve
from modeling_helper import transform_xy
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
flights = pd.read_csv('/Users/user/Documents/Github/flightstatus/data/dev/flights_processed.csv')

The dataset includes US flights in 2015. We will use flights 
- from January to June for training
- from July to September for validation
- October to December for test.

In [3]:
datatrain = flights[flights.MONTH<=6]
dataval = flights[(flights.MONTH>=7)&(flights.MONTH<=9)]
datatest = flights[flights.MONTH<=10]

In [4]:
X_train, y_train = transform_xy(datatrain)
X_val, y_val = transform_xy(dataval)
X_test, y_test = transform_xy(datatest)

In [5]:
n_cols = X_train.shape[1]

In [6]:
ss = StandardScaler()
X_train_s = ss.fit_transform(X_train)
X_val_s = ss.transform(X_val)
X_test_s = ss.transform(X_test)

In [7]:
# set up early stopping and class weight
early_stopping_monitor = EarlyStopping(patience=5)
spw = (sum(y_train==0) + sum(y_val==0))/(sum(y_train==1)+sum(y_val==1))
cw = {0:1., 1:spw[1]} # This gives the positive class 1.9 times the weight of the negative class

### Starting model

In [15]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(Dropout(rate = 0.8))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(curve='PR')])
model.fit(X_train_s, y_train, callbacks = [early_stopping_monitor], validation_data=(X_val_s, y_val),
         batch_size=64, class_weight=cw, epochs=50)

Train on 2889506 samples, validate on 1496200 samples
Epoch 1/50
2889506/2889506 [==============================] - 132s 46us/step - loss: 0.9035 - auc_3: 0.5411 - val_loss: 0.6998 - val_auc_3: 0.5541
Epoch 2/50
2889506/2889506 [==============================] - 131s 45us/step - loss: 0.8862 - auc_3: 0.5551 - val_loss: 0.6909 - val_auc_3: 0.5636
Epoch 3/50
2889506/2889506 [==============================] - 133s 46us/step - loss: 0.8813 - auc_3: 0.5662 - val_loss: 0.6863 - val_auc_3: 0.5720
Epoch 4/50
2889506/2889506 [==============================] - 140s 48us/step - loss: 0.8783 - auc_3: 0.5741 - val_loss: 0.6908 - val_auc_3: 0.5774
Epoch 5/50
2889506/2889506 [==============================] - 131s 45us/step - loss: 0.8762 - auc_3: 0.5783 - val_loss: 0.6946 - val_auc_3: 0.5806
Epoch 6/50
2889506/2889506 [==============================] - 118s 41us/step - loss: 0.8745 - auc_3: 0.5808 - val_loss: 0.6957 - val_auc_3: 0.5827
Epoch 7/50
2889506/2889506 [==============================] - 11

__Note__: `class_weight` is used for weighting the loss function during training only. Therefore, we see that the validation loss is lower than the train loss.

In [19]:
y_train_prob = model.predict(X_train_s)[:,1]
y_val_prob = model.predict(X_val_s)[:,1]

In [22]:
precision, recall, thresholds = precision_recall_curve(y_train[:,1], y_train_prob)
auc(recall, precision)

0.5043923450759831

In [23]:
precision, recall, thresholds = precision_recall_curve(y_val[:,1], y_val_prob)
auc(recall, precision)

0.476082238189684

### Increase model capacity
- adding 2 fully connected layers with 100 nodes

In [24]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(Dropout(rate = 0.8))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(curve='PR')])
model.fit(X_train_s, y_train, callbacks = [early_stopping_monitor], validation_data=(X_val_s, y_val),
         batch_size=64, class_weight=cw, epochs=50)

Train on 2889506 samples, validate on 1496200 samples
Epoch 1/50
2889506/2889506 [==============================] - 157s 54us/step - loss: 0.9007 - auc_6: 0.5335 - val_loss: 0.6872 - val_auc_6: 0.5565
Epoch 2/50
2889506/2889506 [==============================] - 142s 49us/step - loss: 0.8877 - auc_6: 0.5613 - val_loss: 0.6811 - val_auc_6: 0.5696
Epoch 3/50
2889506/2889506 [==============================] - 140s 48us/step - loss: 0.8819 - auc_6: 0.5732 - val_loss: 0.6810 - val_auc_6: 0.5779
Epoch 4/50
2889506/2889506 [==============================] - 148s 51us/step - loss: 0.8787 - auc_6: 0.5799 - val_loss: 0.6773 - val_auc_6: 0.5834
Epoch 5/50
2889506/2889506 [==============================] - 122s 42us/step - loss: 0.8764 - auc_6: 0.5850 - val_loss: 0.6771 - val_auc_6: 0.5876
Epoch 6/50
2889506/2889506 [==============================] - 121s 42us/step - loss: 0.8745 - auc_6: 0.5888 - val_loss: 0.6823 - val_auc_6: 0.5904
Epoch 7/50
2889506/2889506 [==============================] - 12

In [25]:
y_train_prob = model.predict(X_train_s)[:,1]
y_val_prob = model.predict(X_val_s)[:,1]

In [26]:
precision, recall, thresholds = precision_recall_curve(y_train[:,1], y_train_prob)
auc(recall, precision)

0.5030946071485047

In [27]:
precision, recall, thresholds = precision_recall_curve(y_val[:,1], y_val_prob)
auc(recall, precision)

0.4804807127748579

In [43]:
model.save('neuralnetwork.h5')

### Increase model capacity
- adding to 500 nodes in the middle layers

In [8]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(Dropout(rate = 0.8))
model.add(Dense(100, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(curve='PR')])
model.fit(X_train_s, y_train, callbacks = [early_stopping_monitor], validation_data=(X_val_s, y_val),
         batch_size=64, class_weight=cw, epochs=50)

Train on 2889506 samples, validate on 1496200 samples
Epoch 1/50
2889506/2889506 [==============================] - 209s 72us/step - loss: 0.8968 - auc: 0.5430 - val_loss: 0.6934 - val_auc: 0.5589
Epoch 2/50
2889506/2889506 [==============================] - 197s 68us/step - loss: 0.8855 - auc: 0.5600 - val_loss: 0.6878 - val_auc: 0.5679
Epoch 3/50
2889506/2889506 [==============================] - 172s 59us/step - loss: 0.8801 - auc: 0.5703 - val_loss: 0.6864 - val_auc: 0.5750
Epoch 4/50
2889506/2889506 [==============================] - 221s 77us/step - loss: 0.8767 - auc: 0.5767 - val_loss: 0.6935 - val_auc: 0.5792
Epoch 5/50
2889506/2889506 [==============================] - 204s 71us/step - loss: 0.8739 - auc: 0.5796 - val_loss: 0.6900 - val_auc: 0.5820
Epoch 6/50
2889506/2889506 [==============================] - 185s 64us/step - loss: 0.8722 - auc: 0.5826 - val_loss: 0.6936 - val_auc: 0.5842
Epoch 7/50
2889506/2889506 [==============================] - 185s 64us/step - loss: 0.8

In [9]:
y_train_prob = model.predict(X_train_s)[:,1]
y_val_prob = model.predict(X_val_s)[:,1]

In [10]:
precision, recall, thresholds = precision_recall_curve(y_train[:,1], y_train_prob)
auc(recall, precision)

0.5052536719926125

In [11]:
precision, recall, thresholds = precision_recall_curve(y_val[:,1], y_val_prob)
auc(recall, precision)

0.4749822763163157

In [12]:
model.save('neuralnetwork.h5')

### Increase model capacity
- adding 2 more fully connected layers of 500 nodes
- increase the middle layer's nodes to 1000

In [32]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(Dropout(rate = 0.8))
model.add(Dense(100, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(curve='PR')])
model.fit(X_train_s, y_train, callbacks = [early_stopping_monitor], validation_data=(X_val_s, y_val),
         batch_size=64, class_weight=cw, epochs=50)

Train on 2889506 samples, validate on 1496200 samples
Epoch 1/50
2889506/2889506 [==============================] - 1435s 497us/step - loss: 0.8994 - auc_8: 0.5332 - val_loss: 0.6851 - val_auc_8: 0.5588
Epoch 2/50
2889506/2889506 [==============================] - 1547s 535us/step - loss: 0.8875 - auc_8: 0.5645 - val_loss: 0.6916 - val_auc_8: 0.5704
Epoch 3/50
2889506/2889506 [==============================] - 1509s 522us/step - loss: 0.8816 - auc_8: 0.5716 - val_loss: 0.6834 - val_auc_8: 0.5769
Epoch 4/50
2889506/2889506 [==============================] - 1632s 565us/step - loss: 0.8780 - auc_8: 0.5789 - val_loss: 0.6822 - val_auc_8: 0.5824
Epoch 5/50
2889506/2889506 [==============================] - 1584s 548us/step - loss: 0.8757 - auc_8: 0.5837 - val_loss: 0.6825 - val_auc_8: 0.5861
Epoch 6/50
2889506/2889506 [==============================] - 1499s 519us/step - loss: 0.8740 - auc_8: 0.5869 - val_loss: 0.6861 - val_auc_8: 0.5887
Epoch 7/50
2889506/2889506 [========================

In [33]:
yy_train_prob = model.predict(X_train_s)[:,1]
y_val_prob = model.predict(X_val_s)[:,1]

In [34]:
precision, recall, thresholds = precision_recall_curve(y_train[:,1], y_train_prob)
auc(recall, precision)

0.5024298687072728

In [35]:
precision, recall, thresholds = precision_recall_curve(y_val[:,1], y_val_prob)
auc(recall, precision)

0.4766038409793225

**The validation loss did not improve in the last round. We will decrease the model capacity by removing a layer.**

In [36]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (n_cols,))) 
model.add(Dropout(rate = 0.8))
model.add(Dense(100, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(lr=0.001), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(curve='PR')])
model.fit(X_train_s, y_train, callbacks = [early_stopping_monitor], validation_data=(X_val_s, y_val),
         batch_size=64, class_weight=cw, epochs=50)

Train on 2889506 samples, validate on 1496200 samples
Epoch 1/50
2889506/2889506 [==============================] - 1061s 367us/step - loss: 0.8984 - auc_9: 0.5411 - val_loss: 0.7021 - val_auc_9: 0.5547
Epoch 2/50
2889506/2889506 [==============================] - 723s 250us/step - loss: 0.8875 - auc_9: 0.5532 - val_loss: 0.7024 - val_auc_9: 0.5608
Epoch 3/50
2889506/2889506 [==============================] - 763s 264us/step - loss: 0.8823 - auc_9: 0.5613 - val_loss: 0.6999 - val_auc_9: 0.5663
Epoch 4/50
2889506/2889506 [==============================] - 933s 323us/step - loss: 0.8790 - auc_9: 0.5673 - val_loss: 0.6967 - val_auc_9: 0.5711
Epoch 5/50
2889506/2889506 [==============================] - 908s 314us/step - loss: 0.8764 - auc_9: 0.5720 - val_loss: 0.6938 - val_auc_9: 0.5749
Epoch 6/50
2889506/2889506 [==============================] - 776s 269us/step - loss: 0.8747 - auc_9: 0.5760 - val_loss: 0.6934 - val_auc_9: 0.5782
Epoch 7/50
2889506/2889506 [=============================

### Make predictions

In [13]:
y_train_prob = model.predict(X_train_s)[:,1]
y_val_prob = model.predict(X_val_s)[:,1]
y_test_prob = model.predict(X_test_s)[:,1]

### ROC

In [16]:
fpr, tpr, thresholds = roc_curve(y_train[:,1], y_train_prob)
auc(fpr, tpr)

0.6633633772331589

In [17]:
fpr, tpr, thresholds = roc_curve(y_val[:,1], y_val_prob)
auc(fpr, tpr)

0.6622402962712145

In [18]:
fpr, tpr, thresholds = roc_curve(y_test[:,1], y_test_prob)
auc(fpr, tpr)

0.6628200490374877

### Precision-Recall Curve

In [19]:
precision, recall, thresholds = precision_recall_curve(y_train[:,1], y_train_prob)
auc(recall, precision)

0.5052536719926125

In [20]:
precision, recall, thresholds = precision_recall_curve(y_val[:,1], y_val_prob)
auc(recall, precision)

0.4749822763163157

In [21]:
precision, recall, thresholds = precision_recall_curve(y_test[:,1], y_test_prob)
auc(recall, precision)

0.49407089543558397

### Reload the final model

In [6]:
#reload the model
m_nn = load_model('neuralnetwork.h5')